In [41]:
import numpy as np
import data
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
%matplotlib inline

In [42]:
train_data = data.Dataset('train.csv')
#train_data.to_one_hot()

In [43]:
pca = PCA(n_components=256)
pca.fit(train_data.x)
train_x = pca.transform(train_data.x)
np.sum(pca.explained_variance_ratio_)


0.9839630084730454

In [44]:
train_x,test_x,train_y,test_y = train_test_split(train_data.x,train_data.y,test_size=0.1)

In [29]:
print(len(train_x))
len(test_x)

1094


122

In [51]:
clf = XGBClassifier(max_depth=3,min_child_weight=3,reg_alpha = 0.1,n_estimators=200)
'''
params = {
    'reg_alpha': [0.1, 1, 10],
}

clf = GridSearchCV(rf, params, cv=5, n_jobs=-1, scoring = 'accuracy',verbose=10)
'''
clf.fit(train_x, train_y)
print("Train acc:",clf.score(train_x,train_y))
clf.score(test_x,test_y)

Train acc: 1.0


0.680327868852459

In [52]:
print(clf.best_score_)
print(clf.best_params_)

AttributeError: 'XGBClassifier' object has no attribute 'best_score_'

In [16]:
test_data = data.Dataset('test.csv')
test_data_x = pca.transform(test_data.x)
pred = np.random.randint(0,20,size=(test_data_x.shape[0]))
data.write_to_csv('submission.csv',pred,train_data.get_cat_to_label())